In [2]:
from threading import *

In [3]:
Thread
Lock
RLock
Condition
Semaphore
BoundedSemaphore
Event
Timer
Barrier

threading.Barrier

In [4]:
def atof():
    for i in 'abcdef':
        print(i)
x = Thread(target=atof)
x.start()

def ftoa():
    for i in 'abcdef'[::-1]:
        print(i)
y = Thread(target=ftoa)
y.start()

a
b
c
d
e
f
f
e
d
c
b
a


In [5]:
# In computer science, a daemon is a process that runs in the background.
x = Thread(target=atof, daemon=True)
x.start()

a
b
c
d
e
f


In [6]:
# join() a Thread
# Daemon threads are handy, but what about when you want to wait for a thread to stop? What about when you want to do that and not exit your program?
def atof():
    for i in 'abcdef':
        print(i)
x = Thread(target=atof)
x.start()

def ftoa():
    for i in 'abcdef'[::-1]:
        print(i)
y = Thread(target=ftoa)
y.start()

x.join()
y.join()

a
b
c
d
e
f
f
e
d
c
b
a


In [7]:
# Using a ThreadPoolExecutor
# There’s an easier way to start up a group of threads than the one you saw above.
# It’s called a ThreadPoolExecutor, and it’s part of the standard library in concurrent.futures
import concurrent.futures
# The code creates a ThreadPoolExecutor as a context manager, telling it how many worker threads it wants in the pool.
# It then uses .map() to step through an iterable of things, in your case range(3), passing each one to a thread in the pool.
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
# concurrent.futures.ProcessPoolExecutor
    executor.map(atof, range(5))
# Unfortunately, ThreadPoolExecutor will hide that exception, and (in the case above) the program terminates with no output.
# This can be quite confusing to debug at first.

In [8]:
# Basic Synchronization Using Lock
# There are a number of ways to avoid or solve race conditions.
lock = Lock()
lock.acquire()
# ...
print('isolation')
lock.release()

isolation


In [11]:
# Threading: multiple-threads, share data between threads, how to use locks to prevent raise conditions, deamon process, how to use queue for thread safe data exchange

from threading import Thread, Lock
import time

database_value = 0

def increase(lock):
    global database_value
        
    lock.acquire()
    # ( some processing
    local_copy = database_value
    local_copy += 1
    time.sleep(0.1) # )
    database_value = local_copy
    lock.release() # Necessary (otherwise we'll stuck here)


lock = Lock()

print("start")


thread1 = Thread(target=increase, args=(lock,))
thread2 = Thread(target=increase, args=(lock,))

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print("end value", database_value)

print("end main")

"""
start
end value 2
end main
"""

start
end value 2
end main


'\nstart\nend value 2\nend main\n'

In [10]:
with lock:
    # ...
    pass

In [8]:
from threading import Thread, current_thread
from queue import Queue

def worker(q):
    while(True):
        value = q.get()
        print(f"in {current_thread().name} got {value}")
        q.task_done()


q = Queue()

num_threads = 10

for i in range(num_threads):
    thread = Thread(target=worker, args=(q,))
    thread.deamon = True # (by default is False)
    thread.start()

for i in range(1, 21):
    q.put(i)

q.join()

print("end main")

in Thread-61 (worker) got 1in Thread-65 (worker) got 2
in Thread-65 (worker) got 3
in Thread-65 (worker) got 4
in Thread-65 (worker) got 5
in Thread-65 (worker) got 6
in Thread-65 (worker) got 7
in Thread-65 (worker) got 8
in Thread-65 (worker) got 9
in Thread-65 (worker) got 10
in Thread-65 (worker) got 11
in Thread-65 (worker) got 12
in Thread-65 (worker) got 13
in Thread-65 (worker) got 14
in Thread-65 (worker) got 15
in Thread-65 (worker) got 16
in Thread-65 (worker) got 17
in Thread-65 (worker) got 18
in Thread-65 (worker) got 19
in Thread-65 (worker) got 20

end main


In [9]:
from threading import Thread, current_thread, Lock
from queue import Queue

def worker(q, lock):
    while(True):
        value = q.get()
        with lock:
            print(f"in {current_thread().name} got {value}")
        q.task_done()


q = Queue()
lock = Lock()
num_threads = 10

for i in range(10):
    thread = Thread(target=worker, args=(q,lock))
    # Deamon-Thread will die when Main-Thread dies
    thread.deamon = True # (by default is False) , deamon thread will die when main thread dies
    thread.start()

for i in range(1, 21):
    q.put(i)

q.join()

print("end main")

in Thread-71 (worker) got 1
in Thread-71 (worker) got 2
in Thread-71 (worker) got 3
in Thread-71 (worker) got 4
in Thread-71 (worker) got 5
in Thread-71 (worker) got 6
in Thread-71 (worker) got 7
in Thread-71 (worker) got 8
in Thread-71 (worker) got 9
in Thread-71 (worker) got 10
in Thread-71 (worker) got 11
in Thread-71 (worker) got 12
in Thread-71 (worker) got 13
in Thread-71 (worker) got 14
in Thread-71 (worker) got 15
in Thread-71 (worker) got 16
in Thread-71 (worker) got 17
in Thread-71 (worker) got 18
in Thread-71 (worker) got 19
in Thread-71 (worker) got 20
end main


In [10]:
from threading import Thread, current_thread, Lock
from queue import Queue

def worker(q, lock):
    while(True):
        value = q.get()
        with lock:
            print(f"in {current_thread().name} got {value}")
        q.task_done()
        
        if q.empty():
            break


q = Queue()
lock = Lock()
num_threads = 10

for i in range(10):
    thread = Thread(target=worker, args=(q,lock))
    
    # Deamon-Thread is False
    thread.deamon = False
    
    thread.start()

for i in range(1, 21):
    q.put(i)

q.join()

print("end main")

in Thread-81 (worker) got 1
in Thread-81 (worker) got 2
in Thread-81 (worker) got 3
in Thread-81 (worker) got 4
in Thread-81 (worker) got 5
in Thread-81 (worker) got 6
in Thread-81 (worker) got 7
in Thread-81 (worker) got 8
in Thread-81 (worker) got 9
in Thread-81 (worker) got 10
in Thread-81 (worker) got 11
in Thread-81 (worker) got 12
in Thread-81 (worker) got 13
in Thread-81 (worker) got 14
in Thread-81 (worker) got 15
in Thread-81 (worker) got 16
in Thread-81 (worker) got 17
in Thread-81 (worker) got 18
in Thread-81 (worker) got 19
in Thread-81 (worker) got 20
end main
